In [ ]:
import numpy as np
import random
import math
import copy
import time

class Solution(object):
    def __init__(self):
        self.path = []
        self.code = []
        self.fitness = float('inf')

class FA:

    def __init__(self, src, dst, N, Max, K_paths, y, a0, b0):
        self.switches = [i for i in range(1,41)]
        self.src= src
        self.dst = dst
        self.weight_map= self.GetWeightMap()
        self.adjacency = copy.deepcopy(self.weight_map)
        self.N = N
        self.Max = Max
        self.K_paths = K_paths
        self.population = [self.CreateSolution() for i in range(self.N)]
        self.y = y
        self.a0 = a0
        self.b0 = b0
        self.a = 0
        self.condidates = []
        self.best = []

    def GetWeightMap(self):
        weight_map={}
        temp = 0
        with open('metric_data_40.txt') as f:
            for line in f:
                strt = line
                strt2 = strt.split(':')
                my_result = list(map(int, strt2[0].split(',')))
                if (temp!=my_result[0]):
                    weight_map[my_result[0]]={}
                weight_map[my_result[0]][my_result[1]] = int(strt2[1])
                temp = my_result[0]
        return weight_map

    def CreateSolution(self):
        newSolution = Solution()
        path=[]
        code = []
        while(len(path)==0):
            code.clear()
            path.clear()
            code = copy.deepcopy([random.uniform(-1,1) for i in range(len(self.switches))])
            path = copy.deepcopy(self.Decode(code))
        newSolution.code = copy.deepcopy(code)
        newSolution.path = copy.deepcopy(path)
        newSolution.fitness = self.Evaluate(newSolution.path)
        return newSolution

    def Decode(self,code):
        path = []
        path.append(self.src)
        current_switch = self.src
        while(current_switch!=self.dst):
            neighbor_switches = set(self.adjacency[current_switch].keys())-set(path)
            neighbor_switches = list(neighbor_switches)
            if(len(neighbor_switches)==0):
                path.clear()
                break
            switch_min = 1
            min = float('inf')
            for switch in neighbor_switches:
                if(code[switch-1] <= min):
                    min = code[switch-1]
                    switch_min = switch
            current_switch = switch_min
            path.append(current_switch)
        return path

    def Evaluate(self,path):
        calculatedFitness = 0
        for i in range(len(path) - 1):
            p1 = path[i]
            p2 = path[i + 1]
            calculatedFitness += self.weight_map[p1][p2]
        return calculatedFitness

    def Normalize(self,code):
        code_2 = copy.deepcopy(code)
        mn = min(code_2)
        mx = max(code_2)
        for j in range(len(self.switches)):
            code[j] = -1+2*(code_2[j]-mn)/(mx-mn)
        return code

    def Attract(self):
        for i in range(self.N):
            for j in range(self.N):
                newSolution = Solution()
                if(self.population[i].fitness > self.population[j].fitness):
                    r2 = 0
                    for j1 in range(len(self.switches)):
                        number = self.population[i].code[j1] - self.population[j].code[j1]
                        r2 += pow(number,2)
                    b = self.b0*math.exp(-self.y*r2)
                    for j2 in range(len(self.switches)):
                        newSolution.code.append(self.population[i].code[j2] + b*(self.population[j].code[j2] - self.population[i].code[j2]))
                    code_2 = [0 for i in range(len(self.switches))]
                    while(len(newSolution.path)==0):
                        for j2 in range(len(self.switches)):
                            e = np.random.rand()-1/2
                            code_2[j2] = newSolution.code[j2] + self.a*2*e
                        code_2 = self.Normalize(code_2)
                        newSolution.path = copy.deepcopy(self.Decode(code_2))
                    newSolution.code = copy.deepcopy(code_2)
                    newSolution.fitness = self.Evaluate(newSolution.path)
                    self.population[i] = copy.deepcopy(newSolution)


    def MemorizeCondidates(self):
        self.population.sort(key=lambda x: x.fitness)
        condidate = []
        k=0
        for i in range(len(self.population)):
            dk_3 = False
            for member in condidate:
                if(tuple(member.path)==tuple(self.population[i].path)):
                    dk_3 = True
                    break
            if(dk_3!=True):
                condidate.append(copy.deepcopy(self.population[i]))
                k=k+1
            if(k==self.K_paths):
                break
        self.condidates.extend(copy.deepcopy(condidate))

    def GetBest(self):
        self.condidates.sort(key=lambda x: x.fitness)
        self.best.clear()
        k=0
        for i in range(len(self.condidates)):
            dk_3 = False
            for solution in self.best:
                if(tuple(solution.path)==tuple(self.condidates[i].path)):
                    dk_3 = True
                    break
            if(dk_3!=True):
                self.best.append(copy.deepcopy(self.condidates[i]))
                k=k+1
            if(k==self.K_paths):
                break

    def Do(self):
        for i in range(self.Max):
            self.a = self.a0*pow(0.99,i)
            self.Attract()
            self.MemorizeCondidates()
        self.GetBest()

In [ ]:
import numpy as np
import random
import math
import copy
import time

class Solution(object):
    def __init__(self):
        self.path = []
        self.code = []
        self.fitness = float('inf')

class MFA:

    def __init__(self, src, dst, N, Max, K_paths, y, a0, b0):
        self.switches = [i for i in range(1,41)]
        self.src= src
        self.dst = dst
        self.weight_map= self.GetWeightMap()
        self.adjacency = copy.deepcopy(self.weight_map)
        self.N = N
        self.Max = Max
        self.K_paths = K_paths
        self.population = [self.CreateSolution() for i in range(self.N)]
        self.y = y
        self.a0 = a0
        self.b0 = b0
        self.a = 0
        self.condidates = []
        self.best = []

    def GetWeightMap(self):
        weight_map={}
        temp = 0
        with open('metric_data_40.txt') as f:
            for line in f:
                strt = line
                strt2 = strt.split(':')
                my_result = list(map(int, strt2[0].split(',')))
                if (temp!=my_result[0]):
                    weight_map[my_result[0]]={}
                weight_map[my_result[0]][my_result[1]] = int(strt2[1])
                temp = my_result[0]
        return weight_map

    def CreateSolution(self):
        newSolution = Solution()
        path=[]
        code = []
        while(len(path)==0):
            code.clear()
            path.clear()
            code = copy.deepcopy([random.uniform(-1,1) for i in range(len(self.switches))])
            path = copy.deepcopy(self.Decode(code))
        newSolution.code = copy.deepcopy(code)
        newSolution.path = copy.deepcopy(path)
        newSolution.fitness = self.Evaluate(newSolution.path)
        return newSolution

    def Decode(self,code):
        path = []
        path.append(self.src)
        current_switch = self.src
        while(current_switch!=self.dst):
            neighbor_switches = set(self.adjacency[current_switch].keys())-set(path)
            neighbor_switches = list(neighbor_switches)
            if(len(neighbor_switches)==0):
                path.clear()
                break
            switch_min = 1
            min = float('inf')
            for switch in neighbor_switches:
                if(code[switch-1] <= min):
                    min = code[switch-1]
                    switch_min = switch
            current_switch = switch_min
            path.append(current_switch)
        return path

    def Evaluate(self,path):
        calculatedFitness = 0
        for i in range(len(path) - 1):
            p1 = path[i]
            p2 = path[i + 1]
            calculatedFitness += self.weight_map[p1][p2]
        return calculatedFitness

    def Normalize(self,code):
        code_2 = copy.deepcopy(code)
        mn = min(code_2)
        mx = max(code_2)
        for j in range(len(self.switches)):
            code[j] = -1+2*(code_2[j]-mn)/(mx-mn)
        return code

    def Attract(self):
        for i in range(self.N):
            for j in range(self.N):
                newSolution = Solution()
                if(self.population[i].fitness > self.population[j].fitness):
                    r2 = 0
                    for j1 in range(len(self.switches)):
                        number = self.population[i].code[j1] - self.population[j].code[j1]
                        r2 += pow(number,2)
                    b = self.b0*math.exp(-self.y*r2)
                    for j2 in range(len(self.switches)):
                        newSolution.code.append(self.population[i].code[j2] + b*(self.population[j].code[j2] - self.population[i].code[j2]))
                    code_2 = [0 for i in range(len(self.switches))]
                    while(len(newSolution.path)==0):
                        for j2 in range(len(self.switches)):
                            e = np.random.rand()-1/2
                            code_2[j2] = newSolution.code[j2] + self.a*2*e
                        code_2 = self.Normalize(code_2)
                        newSolution.path = copy.deepcopy(self.Decode(code_2))
                    newSolution.code = copy.deepcopy(code_2)
                    newSolution.fitness = self.Evaluate(newSolution.path)
                    if(self.population[i].fitness > newSolution.fitness):
                        self.population[i] = copy.deepcopy(newSolution)
                    break


    def MemorizeCondidates(self):
        self.population.sort(key=lambda x: x.fitness)
        condidate = []
        k=0
        for i in range(len(self.population)):
            dk_3 = False
            for member in condidate:
                if(tuple(member.path)==tuple(self.population[i].path)):
                    dk_3 = True
                    break
            if(dk_3!=True):
                condidate.append(copy.deepcopy(self.population[i]))
                k=k+1
            if(k==self.K_paths):
                break
        self.condidates.extend(copy.deepcopy(condidate))

    def GetBest(self):
        self.condidates.sort(key=lambda x: x.fitness)
        self.best.clear()
        k=0
        for i in range(len(self.condidates)):
            dk_3 = False
            for solution in self.best:
                if(tuple(solution.path)==tuple(self.condidates[i].path)):
                    dk_3 = True
                    break
            if(dk_3!=True):
                self.best.append(copy.deepcopy(self.condidates[i]))
                k=k+1
            if(k==self.K_paths):
                break

    def Do(self):
        for i in range(self.Max):
            self.a = 0.9 + self.a0*pow(0.99,i)
            self.Attract()
            self.MemorizeCondidates()
        self.GetBest()

In [ ]:
Times = 5
N = 20
Max = 1000
K_paths = 10
y = 1
a0 = 1
b0 = 1

In [ ]:
from YenAlgorithm import YenAlgorithm
weight_map={}
temp = 0
with open('metric_data_40.txt') as f:
    for line in f:
        strt = line
        strt2 = strt.split(':')
        my_result = list(map(int, strt2[0].split(',')))
        if (temp!=my_result[0]):
            weight_map[my_result[0]]={}
        weight_map[my_result[0]][my_result[1]] = int(strt2[1])
        temp = my_result[0]
vertices = [i for i in range(1,41)]
alg_y = YenAlgorithm(weight_map,vertices,1,10,K_paths)
paths_vertices = alg_y.compute_shortest_paths()

In [ ]:
paths_length = []
for path in paths_vertices:
    s = 0
    for i in range(len(path)-1):
        s+= weight_map[path[i]][path[i+1]]
    paths_length.append(s)
print(paths_length)
paths_vertices

[185, 185, 195, 195, 210, 210, 210, 215, 215, 220]


[[1, 18, 19, 20, 21, 22, 23, 24, 10],
 [1, 11, 12, 13, 14, 15, 16, 17, 10],
 [1, 25, 26, 27, 28, 29, 30, 31, 32, 10],
 [1, 2, 3, 4, 5, 14, 15, 16, 17, 10],
 [1, 33, 34, 35, 36, 37, 38, 39, 40, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 21, 22, 23, 24, 10],
 [1, 2, 3, 4, 5, 6, 7, 16, 17, 10],
 [1, 2, 3, 4, 5, 13, 14, 15, 16, 17, 10],
 [1, 11, 12, 13, 14, 15, 30, 31, 32, 10]]

In [ ]:
sum(paths_length)

2040

In [ ]:
solan_1 = np.zeros(K_paths)
solan_2 = np.zeros(K_paths)
CD_1 = []
CD_2 = []
time_1 = []
time_2 = []
for i in range(Times):
    print("iteration: ", i)
    alg_1 = FA(1,10,N, Max, K_paths, y, a0, b0)
    start_1 = time.time()
    alg_1.Do()
    end_1 = time.time()
    time_1.append(end_1 - start_1)
    for j in range(len(paths_vertices)):
        for j2 in range(len(alg_1.best)):
            if(tuple(alg_1.best[j2].path)==tuple(paths_vertices[j])):
                solan_1[j]+=1
                break
    s = 0
    for member in alg_1.best:
        for i in range(len(member.path)-1):
            s+= alg_1.weight_map[member.path[i]][member.path[i+1]]
    CD_1.append(s)

#     alg_2 = MFA(1,10,N, Max, K_paths, y, a0, b0)
#     start_2 = time.time()
#     alg_2.Do()
#     end_2 = time.time()
#     time_2.append(end_2 - start_2)
#     for j in range(len(paths_vertices)):
#         for j2 in range(len(alg_2.best)):
#             if(tuple(alg_2.best[j2].path)==tuple(paths_vertices[j])):
#                 solan_2[j]+=1
#                 break
#     s = 0
#     for member in alg_2.best:
#         for i in range(len(member.path)-1):
#             s+= alg_2.weight_map[member.path[i]][member.path[i+1]]
#     CD_2.append(s)

iteration:  0
iteration:  1
iteration:  2
iteration:  3


In [ ]:
len(CD_1)

In [ ]:
print(solan_1)
print(solan_2)

In [ ]:
time_1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
font = {'size'   : 11}
matplotlib.rc('font', **font)
# create data
x = np.arange(K_paths)+1
width = 0.3

# plot data in grouped manner of bar type
fig = plt.figure(figsize=(10,5))
plt.bar(x-0.15, solan_1, width, color='red')
plt.bar(x+0.15, solan_2, width, color='springgreen', hatch="//")
plt.xlabel("Path No.")
plt.ylabel("Number of successful searches")
plt.legend(["FA", "MFA"],loc="upper right")
plt.grid()
plt.savefig("FA/FA_MFA_40.png",dpi=200)

In [ ]:
figg1 = plt.figure(figsize=(10,5))
avr_1 = int(sum(CD_1)/Times)
avr_2 = int(sum(CD_2)/Times)
plt.plot(CD_1, label = "FA" + " (Average = "+str(avr_1)+")", linewidth = '3', color='red')
plt.plot(CD_2, label = "MFA" + " (Average = "+str(avr_2)+")", linewidth = '3', color='springgreen', linestyle = 'dashed')
plt.legend(loc="upper right")
plt.xlabel("Test No.")
plt.ylabel("CD")
plt.grid()
plt.savefig("FA/FA_MFA_40_CD.png",dpi=200)

In [ ]:
figg1 = plt.figure(figsize=(10,5))
avr_1 = round(sum(time_1)/Times,2)
avr_2 = round(sum(time_2)/Times,2)
plt.plot(time_1, label = "FA" + " (Average = "+str(avr_1)+"s)", linewidth = '3', color='red')
plt.plot(time_2, label = "MFA" + " (Average = "+str(avr_2)+"s)", linewidth = '3', color='springgreen', linestyle = 'dashed')
plt.legend(loc="upper right")
plt.xlabel("Test No.")
plt.ylabel("Time in seconds")
plt.grid()
plt.savefig("FA/FA_MFA_40_time.png",dpi=200)